# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Hoàng Đức Việt
- MSSV: 21127203
- Lớp: 21CLC02

# Import

In [21]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [22]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

- Chuyển dữ liệu thành `numpy array`:

In [23]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

# Cài đặt hàm

In [24]:
# Cài đặt các hàm cần thiết ở đây

# Tìm giá trọng số của w
def find_weight(X, y):
    return np.round(np.linalg.inv(X.T @ X) @ X.T @ y, 3)

# Tính giá trị dự đoán của y
def calc_y_predict(X, w):
    return np.sum(X * w.T, axis = 1)
    
# Tính độ lỗi MAE giữa các giá trị dự đoán và giá trị thực tế
def MAE(y_predict, y_real):
    return np.mean(np.abs(y_predict - y_real))

def create_result_table(feature, MAE, column_name):
    return pd.DataFrame(zip(feature, MAE), columns = column_name)

- Tìm mô hình tốt nhất:

In [25]:
# Tìm mô hình tốt nhất 
# table là dataframe
def find_best_model(table, feature, clusters = 10):
    # Lấy thêm cột 'Salary' để giữ giá trị y khi xáo trộn
    feature.append("Salary")

    # Lấy bảng dữ liệu sau khi được xáo trộn các dòng
    table_np = table.loc[:, feature].sample(frac = 1).to_numpy()
    
    # Xóa 'Salary' ra khỏi các đặc trưng
    feature.remove("Salary")

    # Chia bảng dữ liệu thành các nhóm
    table_cluster = np.array_split(table_np, clusters)

    # Tìm min MAE trung bình của mỗi nhóm
    MAE_avg = np.zeros(len(feature))

    for c in range(clusters):
        for f in range(len(feature)):
            w_temporary = find_weight(table_cluster[c][:, f, None], table_cluster[c][:, -1, None])

            for i in range(clusters):
                if (c != i):
                    MAE_avg[f] += MAE(calc_y_predict(table_cluster[i][:, f, None], w_temporary), table_cluster[i][:, -1])

    MAE_avg /= clusters * (clusters - 1)
    return np.argmin(MAE_avg), MAE_avg

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [26]:
# Phần code cho yêu cầu 1a
X_train_1a = X_train_np[:, :11]
w = find_weight(X_train_1a, y_train_np)

In [27]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
X_test_1a = X_test_np[:, :11]
y_predict = calc_y_predict(X_test_1a, w)
print("Weight:", w)
print("MAE:", MAE(y_predict, y_test))

Weight: [-22756.513    804.503   1294.655 -91781.898  23182.389   1437.549
  -8570.662    147.858    152.888    117.222  34552.286]
MAE: 104863.71390646267


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)


\begin{aligned}
\text{Salary} = & -22756.513 \times Gender &+& 804.503 \times 10percentage &+& 1294.655 \times 12percentage \\
                & + -91781.898 \times CollegeTier &+& 23182.389 \times Degree &+& 1437.549 \times collegeGPA \\
                & + -8570.662 \times CollegeCityTier &+& 147.858 \times English &+& 152.888 \times Logical \\
                & + 117.222 \times Quant &+& 34552.286 \times Domain
\end{aligned}

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [28]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

feature = ["conscientiousness", "agreeableness", "extraversion", "nueroticism", "openess_to_experience"]

best_personality_feature_index, MAE_avg = find_best_model(train, feature)
best_personality_feature = feature[best_personality_feature_index]

print(create_result_table(feature, MAE_avg, ["Feature", "MAE Average"]))
print("Best feature:", best_personality_feature)

                 Feature    MAE Average
0      conscientiousness  306744.380974
1          agreeableness  300658.964486
2           extraversion  306997.207427
3            nueroticism  299460.994996
4  openess_to_experience  303200.492350
Best feature: nueroticism


In [29]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
def best_personality_feature_model(table, best_personality_feature):
    X = table.loc[:, best_personality_feature].to_numpy()
    y = table.iloc[:, -1].to_numpy()
    return find_weight(X[:, None], y[:, None])

In [30]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
w = best_personality_feature_model(train, best_personality_feature)
y_predict = calc_y_predict(test.loc[:, best_personality_feature].to_numpy()[:, None], w)
print("Weight:", w)
print("MAE:", MAE(y_predict, y_test_np))

Weight: [[-56546.304]]
MAE: 291019.6931941854


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -56546.304 \times nueroticism$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [31]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

feature = ["English", "Logical", "Quant"]

best_personality_feature_index, MAE_avg = find_best_model(train, feature)
best_personality_feature = feature[best_personality_feature_index]

print(create_result_table(feature, MAE_avg, ["Feature", "MAE Average"]))
print("Best feature:", best_personality_feature)

   Feature    MAE Average
0  English  122701.902788
1  Logical  121256.765978
2    Quant  118994.415870
Best feature: Quant


In [32]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
def best_skill_feature_model(table, best_personality_feature):
    X = table.loc[:, best_personality_feature].to_numpy()
    y = table.iloc[:, -1].to_numpy()
    return find_weight(X[:, None], y[:, None])

In [33]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
w = best_personality_feature_model(train, best_personality_feature)
y_predict = calc_y_predict(test.loc[:, best_personality_feature].to_numpy()[:, None], w)
print("Weight:", w)
print("MAE:", MAE(y_predict, y_test_np))

Weight: [[585.895]]
MAE: 106819.53989333333


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 585.895 \times Quant$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [34]:
# Trình bày các phần tìm ra mô hình

exponent = 2

# Mô hình 1: Brute Force với mũ là 2
model1 = ["10percentage", "12percentage", "CollegeTier", "collegeGPA", "CollegeCityTier", "Quant", "Domain", "ComputerProgramming", "ElectronicsAndSemicon", "ComputerScience", "ElectricalEngg", "CivilEngg", "conscientiousness", "extraversion", "nueroticism", "openess_to_experience"]

# Mô hình 2: Lấy các đặc trưng sao cho các giá trị của từng đặc trưng là dàn trải
# Ví dụ: Bỏ qua 'Gender' vì đặc trưng này chỉ có 2 giá trị
model2 = ["10percentage", "12percentage", "collegeGPA", "English", "Logical", "Quant", "Domain", "ComputerProgramming", "ElectronicsAndSemicon", "ComputerScience", "MechanicalEngg", "ElectricalEngg", "TelecomEngg", "CivilEngg", "conscientiousness", "agreeableness", "extraversion", "nueroticism", "openess_to_experience"]

# Mô hình 3: Tương tự như mô hình 2 nhưng bỏ các đặc trưng không phân hóa rộng 
# Ví dụ: Bỏ 'CivilEng' vì đặc trưng này chỉ có giá trị 0 là nổi bật, 
#        còn các giá trị khác phân hóa không rộng
model3 = ["10percentage", "12percentage", "collegeGPA", "English", "Logical", "Quant", "conscientiousness", "agreeableness", "extraversion", "nueroticism", "openess_to_experience"]\

# Thêm các mô hình vào mảng 'model'
model = [model1, model2, model3]

## Thử nghiệm, so sánh các mô hình

- Tạo ra mảng mask gồm các giá trị True/False để xử lí trên numpy.array:

In [35]:
def get_masked_array(table, feature):
    column_name = list(table.columns)
    return np.array([1 if c in feature else 0 for c in column_name], dtype = bool)

In [36]:
# Cập nhật các model thành masked array

for i in range(len(model)):
    model[i] = get_masked_array(train, model[i])

In [37]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

def find_best_model_1d(table, model, clusters = 10, exponent = 2):
    # Lấy bảng dữ liệu sau khi được chia thành các nhóm
    table_np = table.sample(frac = 1).to_numpy()
    table_cluster = np.array_split(table_np, clusters)

    # Khai báo mảng MAE trung bình
    MAE_avg = np.zeros(len(model))

    # Tính MAE trung bình cho mỗi mô hình
    for c in range(clusters):
        for m in range(len(model)):
            w_temporary = find_weight(table_cluster[c][:, m, None] ** exponent, table_cluster[c][:, -1, None])

            for i in range(clusters):
                if (c != i):
                    MAE_avg[m] += MAE(calc_y_predict(table_cluster[i][:, m, None] ** exponent, w_temporary), table_cluster[i][:, -1])

    MAE_avg /= clusters * (clusters - 1)
    return np.argmin(MAE_avg), MAE_avg

In [38]:
# Tìm ra mô hình tốt nhất
best_model_index, MAE_avg = find_best_model_1d(train, model)
print(create_result_table(["Model 1", "Model 2", "Model 3"], MAE_avg, ["Model", "MAE"]))

     Model            MAE
0  Model 1  200854.243426
1  Model 2  121868.114294
2  Model 3  125646.245966


In [39]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

def my_best_model(table, model, exponent = 2):
    X = table.iloc[:, model].to_numpy()
    y = table.iloc[:, -1].to_numpy()
    return find_weight(X ** exponent, y[:, None])

In [40]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
w = my_best_model(train, model[best_model_index])
y_predict = calc_y_predict(test.iloc[:, model[best_model_index]].to_numpy() ** exponent, w)
print("Weight:", w)
print("MAE:", MAE(y_predict, y_test_np))

Weight: [[   155.539]
 [   815.713]
 [   289.421]
 [   138.827]
 [    80.259]
 [   160.23 ]
 [ 26769.152]
 [    79.349]
 [   -45.498]
 [  -178.591]
 [    48.354]
 [  -140.322]
 [   -67.954]
 [   135.797]
 [-21815.3  ]
 [ 18416.53 ]
 [  3705.96 ]
 [-13539.38 ]
 [ -6115.718]]
MAE: 103903.34241398386


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

\begin{aligned}
\text{Salary} 
&=& 155.539 \times 10percentage &+& 815.713 \times 12percentage &+& 289.421 \times collegeGPA \\
&+& 138.827 \times English &+& 80.259 \times Logical &+& 160.23 \times Quant \\
&+& 26769.152 \times Domain &+& 79.349 \times ComputerProgramming &+& -45.498 \times ElectronicsAndSemicon \\
&+& -178.591 \times ComputerScience &+& 48.354 \times MechanicalEngg &+& -140.322 \times ElectricalEngg \\
&+& -67.954 \times TelecomEngg &+& 135.797 \times CivilEngg &+& -21815.3 \times conscientiousness \\
&+& 18416.53 \times agreeableness &+& 3705.96 \times extraversion &+& -13539.38 \times nueroticism \\
&+& -6115.718 \times openess\_to\_experience
\end{aligned}